In [1]:
import sys
from pathlib import Path
import h5py
from argparse import ArgumentParser

import numpy as np
import torch
from sklearn.ensemble import RandomForestClassifier
from cropharvest.datasets import CropHarvest, CropHarvestLabels, Task
from cropharvest.columns import NullableColumns, RequiredColumns
from cropharvest.engineer import Engineer
from cropharvest.bands import BANDS, DYNAMIC_BANDS, STATIC_BANDS, REMOVED_BANDS

sys.path.append("..")

from src.models import STR2MODEL, STR2BASE, train_model

In [2]:
DATA_DIR = "../data/cropharvest"

In [3]:
# This will download all geowiki data
evaluation_datasets = CropHarvest.create_benchmark_datasets(DATA_DIR)

## Get geowiki data

- Load all labels and get all h5 files paths
- Recalculate normalizing dict
- Get as array

In [4]:
class GeowikiCropHarvest():
    def __init__(self, root="data"):    
        self.root = Path(root)
        cropharvest_labels = CropHarvestLabels(root, download=True)
        cropharvest_df = cropharvest_labels.as_geojson()
        self.labels = cropharvest_df[cropharvest_df['dataset'] == 'geowiki-landcover-2017'].reset_index(drop=True)
        self._discard_missing_files()

        self.filepaths = self.labels['path'].tolist()
        self.y_vals = self.labels['is_crop'].tolist()
        self.normalizing_dict = None # TODO: recalculate

    def __len__(self):
        return len(self.filepaths)

    def __getitem__(self, index: int):
        file = h5py.File(self.filepaths[index], "r")
        return self._normalize(file.get("array")[:]), self.y_vals[index]
    
    def as_array(self, flatten_x=False):
        indices_to_sample = list(range(len(self)))
        X, Y = zip(*[self[i] for i in indices_to_sample])
        X_np, y_np = np.stack(X), np.stack(Y)
        if flatten_x:
            X_np = self._flatten_array(X_np)
        return X_np, y_np

    def _path_from_row(self, row):
        path = self.root / f"features/arrays/{row[RequiredColumns.INDEX]}_{row[RequiredColumns.DATASET]}.h5"
        if not path.exists():
            return None
        return path

    def _discard_missing_files(self):
        self.labels['path'] = self.labels.apply(lambda row: self._path_from_row(row), axis=1)
        self.labels = self.labels[~self.labels['path'].isna()].reset_index(drop=True)

    def _normalize(self, array):
        if not self.normalizing_dict:
            return array
        return (array - self.normalizing_dict["mean"]) / self.normalizing_dict["std"]
            
    @staticmethod
    def _flatten_array(array):
        return array.reshape(array.shape[0], -1)

dataset = GeowikiCropHarvest(DATA_DIR)    

In [5]:
dataset.labels

,harvest_date,planting_date,label,classification_label,index,is_crop,lat,lon,dataset,collection_date,export_end_date,externally_contributed_dataset,is_test,geometry,path
0,None,None,None,None,0,0,-16.547619,46.250000,geowiki-landcover-2017,2016-09-30T00:00:00,2017-02-01T00:00:00,False,False,POINT (46.25000 -16.54762),../data/cropharvest/features/arrays/0_geowiki-...
1,None,None,None,None,1,1,-18.547619,48.250000,geowiki-landcover-2017,2016-09-30T00:00:00,2017-02-01T00:00:00,False,False,POINT (48.25000 -18.54762),../data/cropharvest/features/arrays/1_geowiki-...
2,None,None,None,None,2,0,-21.547619,44.250000,geowiki-landcover-2017,2016-09-30T00:00:00,2017-02-01T00:00:00,False,False,POINT (44.25000 -21.54762),../data/cropharvest/features/arrays/2_geowiki-...
3,None,None,None,None,3,1,-17.547619,45.250000,geowiki-landcover-2017,2016-09-30T00:00:00,2017-02-01T00:00:00,False,False,POINT (45.25000 -17.54762),../data/cropharvest/features/arrays/3_geowiki-...
4,None,None,None,None,4,0,-21.547619,46.250000,geowiki-landcover-2017,2016-09-30T00:00:00,2017-02-01T00:00:00,False,False,POINT (46.25000 -21.54762),../data/cropharvest/features/arrays/4_geowiki-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24756,None,None,None,None,35849,1,16.651786,103.550595,geowiki-landcover-2017,2016-09-30T00:00:00,2017-02-01T00:00:00,False,False,POINT (103.55060 16.65179),../data/cropharvest/features/arrays/35849_geow...
24757,None,None,None,None,35850,1,22.651786,84.550595,geowiki-landcover-2017,2016-09-30T00:00:00,2017-02-01T00:00:00,False,False,POINT (84.55060 22.65179),../data/cropharvest/features/arrays/35850_geow...
24758,None,None,None,None,35851,1,10.651786,76.550595,geowiki-landcover-2017,2016-09-30T00:00:00,2017-02-01T00:00:00,False,False,POINT (76.55060 10.65179),../data/cropharvest/features/arrays/35851_geow...
24759,None,None,None,None,35860,0,-0.348214,36.550595,geowiki-landcover-2017,2016-09-30T00:00:00,2017-02-01T00:00:00,False,False,POINT (36.55060 -0.34821),../data/cropharvest/features/arrays/35860_geow...


In [6]:
X, y = dataset.as_array(flatten_x=False)
X_flat, y_flat = dataset.as_array(flatten_x=True)

In [7]:
print(X.shape, y.shape)
X_flat.shape, y_flat.shape

(24761, 12, 18) (24761,)


((24761, 216), (24761,))

## Get Nigeria data

# Train model and evaluate on Nigeria

## 1. Use all data and geowiki labels for training (no validation set)

- Train random forest on it as in demo.ipynb
- Normalizing dict
- Test on Nigeria set (is_test)

### Random forest

In [8]:
model = RandomForestClassifier(random_state=0)
model.fit(X_flat, y_flat)

RandomForestClassifier(random_state=0)

In [9]:
# Predict on training set (just a quick check)
model.predict_proba(X_flat[:10])

array([[0.86, 0.14],
       [0.05, 0.95],
       [0.86, 0.14],
       [0.14, 0.86],
       [0.78, 0.22],
       [0.18, 0.82],
       [0.83, 0.17],
       [0.19, 0.81],
       [0.87, 0.13],
       [0.75, 0.25]])

In [10]:
y_flat[:10]

array([0, 1, 0, 1, 0, 1, 0, 1, 0, 0])

### LSTM model

In [15]:
BANDS

['VV',
 'VH',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'temperature_2m',
 'total_precipitation',
 'elevation',
 'slope',
 'NDVI']

In [11]:
parser = ArgumentParser()

model_args = STR2MODEL["land_cover"].add_model_specific_args(parser).parse_args(args=[])
model = STR2MODEL["land_cover"](model_args)

Checking for data files


35599it [00:09, 3790.58it/s]


All pickle files were found!
length labels: 35599
length pickle files: 35599
length local ids: 490
Found normalizing dict normalizing_dict.pkl
Loading normalizing dict.
{'mean': array([0.19353804, 0.17112217, 0.16083624, 0.16354993, 0.18635676,
       0.25554994, 0.29061711, 0.28009877, 0.31469831, 0.10141977,
       0.0087153 , 0.22964706, 0.15255525, 0.3221835 ]), 'std': array([0.14932182, 0.15265479, 0.14360899, 0.16329558, 0.15796025,
       0.14746618, 0.15011357, 0.14306833, 0.14913972, 0.09338568,
       0.02771975, 0.1111936 , 0.09549155, 0.23958353])}
Train split
Checking for data files


35599it [00:14, 2396.30it/s]


All pickle files were found!
length labels: 28479
length pickle files: 28479
length local ids: 404
{'mean': array([0.19353804, 0.17112217, 0.16083624, 0.16354993, 0.18635676,
       0.25554994, 0.29061711, 0.28009877, 0.31469831, 0.10141977,
       0.0087153 , 0.22964706, 0.15255525, 0.3221835 ]), 'std': array([0.14932182, 0.15265479, 0.14360899, 0.16329558, 0.15796025,
       0.14746618, 0.15011357, 0.14306833, 0.14913972, 0.09338568,
       0.02771975, 0.1111936 , 0.09549155, 0.23958353])}
Val split
Checking for data files


35599it [00:04, 7860.07it/s]


All pickle files were found!
length labels: 7120
length pickle files: 7120
length local ids: 86
{'mean': array([0.19353804, 0.17112217, 0.16083624, 0.16354993, 0.18635676,
       0.25554994, 0.29061711, 0.28009877, 0.31469831, 0.10141977,
       0.0087153 , 0.22964706, 0.15255525, 0.3221835 ]), 'std': array([0.14932182, 0.15265479, 0.14360899, 0.16329558, 0.15796025,
       0.14746618, 0.15011357, 0.14306833, 0.14913972, 0.09338568,
       0.02771975, 0.1111936 , 0.09549155, 0.23958353])}
training set -> number of instances of geowiki_landcover_2017: 28479
training set -> number of instances of nigeria: 915
Total number of files used for training: 29394
Number of model parameters: 23937


In [12]:
model_args

Namespace(add_geowiki=True, add_nigeria=True, add_togo=False, alpha=10, batch_size=64, data_folder='/home/gajo/code/togo-crop-mask/notebooks/../data', geowiki_subset='world', hidden_vector_size=64, learning_rate=0.001, lstm_dropout=0.2, model_base='lstm', multi_headed=False, num_classification_layers=2, num_lstm_layers=1, probability_threshold=0.5, remove_b1_b10=True, weighted_loss_fn=False)

In [19]:
model.input_size = X.shape[-1]

In [20]:
model.base = STR2BASE[model.hparams.model_base](input_size=model.input_size, hparams=model.hparams)

In [21]:
X_tensor = torch.from_numpy(X).to(torch.float32)

In [25]:
y_pred = model.forward(X_tensor)
y_pred, y_pred.shape

(tensor([[0.5180],
         [0.5043],
         [0.5329],
         ...,
         [0.5439],
         [0.5399],
         [0.5115]], grad_fn=<SigmoidBackward0>),
 torch.Size([24761, 1]))

In [33]:
print("Without training, just random weights intialized near 0, this is what the networks predicts as \% of cropland pixels for Geowiki. Makes sense that sigmoid outputs are near 0.5 as sigmoid is 0-centered.")
(y_pred > 0.5).sum() / len(y_pred)

Without training, just random weights intialized near 0, this is what the networks predicts as \% of cropland pixels for Geowiki. Makes sense that sigmoid outputs are near 0.5 as sigmoid is 0-centered.


tensor(0.9196)